In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [3]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "citibike_tabular_data_28d.parquet")
df.head()



,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,target,pickup_hour,pickup_location_id
0,1,0,0,0,0,0,0,0,0,0,...,0,0,5,12,15,12,4,6,2023-01-25 11:00:00,5329.03
1,0,0,0,0,0,0,0,0,0,0,...,0,1,5,12,21,6,8,8,2023-01-26 11:00:00,5329.03
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,10,12,6,8,3,2023-01-27 11:00:00,5329.03
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,6,5,10,23,2023-01-28 11:00:00,5329.03
4,15,12,10,24,23,14,12,6,5,5,...,0,0,0,1,2,9,6,14,2023-01-29 11:00:00,5329.03


In [4]:
df_est = df.copy()
df_est["pickup_hour"] = pd.to_datetime(df_est["pickup_hour"])
df_est["pickup_hour"] = df_est["pickup_hour"].dt.tz_localize("UTC").dt.tz_convert("America/New_York")


In [5]:
df.head()

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,target,pickup_hour,pickup_location_id
0,1,0,0,0,0,0,0,0,0,0,...,0,0,5,12,15,12,4,6,2023-01-25 11:00:00,5329.03
1,0,0,0,0,0,0,0,0,0,0,...,0,1,5,12,21,6,8,8,2023-01-26 11:00:00,5329.03
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,10,12,6,8,3,2023-01-27 11:00:00,5329.03
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,6,5,10,23,2023-01-28 11:00:00,5329.03
4,15,12,10,24,23,14,12,6,5,5,...,0,0,0,1,2,9,6,14,2023-01-29 11:00:00,5329.03


In [6]:
df_est.head()

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,target,pickup_hour,pickup_location_id
0,1,0,0,0,0,0,0,0,0,0,...,0,0,5,12,15,12,4,6,2023-01-25 06:00:00-05:00,5329.03
1,0,0,0,0,0,0,0,0,0,0,...,0,1,5,12,21,6,8,8,2023-01-26 06:00:00-05:00,5329.03
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,10,12,6,8,3,2023-01-27 06:00:00-05:00,5329.03
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,6,5,10,23,2023-01-28 06:00:00-05:00,5329.03
4,15,12,10,24,23,14,12,6,5,5,...,0,0,0,1,2,9,6,14,2023-01-29 06:00:00-05:00,5329.03


In [7]:
from datetime import datetime
from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2023, 9, 1, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(657, 674)
(657,)
(366, 674)
(366,)


In [8]:
import numpy as np

class BaselineModelPreviousHour:
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        return X_test["rides_t-1"]


In [9]:
model = BaselineModelPreviousHour()
predictions = model.predict(X_test)


In [10]:
from sklearn.metrics import mean_absolute_error

test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")


5.7459


In [11]:
from dotenv import load_dotenv
import os

# Absolute path to the .env file
dotenv_path = r"C:\Users\VAISHAK\OneDrive\Desktop\citibikenyc\.env"
result = load_dotenv(dotenv_path)

print("Env file loaded:", result)

# Check if the environment variable is loaded
print("MLFLOW_TRACKING_URI =", os.getenv("MLFLOW_TRACKING_URI"))


Env file loaded: True
MLFLOW_TRACKING_URI = https://dagshub.com/vaishak7026/citibikenyc.mlflow


In [12]:
from src import experiment_utils
from src.experiment_utils import set_mlflow_tracking, log_model_to_mlflow
from dotenv import load_dotenv
load_dotenv() 

mlflow = set_mlflow_tracking()


INFO:src.experiment_utils:MLflow tracking URI and credentials set.


In [13]:
import os
from dotenv import load_dotenv

load_dotenv()

# Debug: check if MLFLOW_TRACKING_URI is available
print("MLFLOW_TRACKING_URI:", os.getenv("MLFLOW_TRACKING_URI"))


MLFLOW_TRACKING_URI: https://dagshub.com/vaishak7026/citibikenyc.mlflow


In [14]:
from dotenv import load_dotenv
load_dotenv()

import os
print(os.getenv("MLFLOW_TRACKING_URI"))  


https://dagshub.com/vaishak7026/citibikenyc.mlflow


In [15]:
import mlflow
print("Tracking URI:", mlflow.get_tracking_uri())


Tracking URI: https://dagshub.com/vaishak7026/citibikenyc.mlflow


In [16]:
from src.experiment_utils import log_model_to_mlflow
from src.experiment_utils import set_mlflow_tracking

log_model_to_mlflow(model, X_test, "BaselineModelPreviousHour", "mean_absolute_error",score=test_mae)

2025/05/10 21:49:19 INFO mlflow.tracking.fluent: Experiment with name 'BaselineModelPreviousHour' does not exist. Creating a new experiment.
INFO:src.experiment_utils:Experiment set to: BaselineModelPreviousHour
INFO:src.experiment_utils:Logged mean_absolute_error: 5.745901639344262
c:\Users\VAISHAK\OneDrive\Desktop\citibikenyc\venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-in

2025/05/10 21:49:23 INFO mlflow.models.model: Found the following environment variables used during model inference: [HOPSWORKS_API_KEY]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
Successfully registered model 'BaselineModelPreviousHour'.
2025/05/10 21:49:29 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: BaselineModelPreviousHour, version 1
Created version '1' of model 'BaselineModelPreviousHour'.
INFO:src.experiment_utils:Model logged with name: BaselineModelPreviousHour


🏃 View run powerful-robin-765 at: https://dagshub.com/vaishak7026/citibikenyc.mlflow/#/experiments/0/runs/ea713c56b29040a285588255103e4b93
🧪 View experiment at: https://dagshub.com/vaishak7026/citibikenyc.mlflow/#/experiments/0


In [17]:
class BaselineModelPreviousWeek:
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        return X_test[f"rides_t-{7*24}"]  # rides_t-168


In [18]:
model = BaselineModelPreviousWeek()
predictions = model.predict(X_test)


In [19]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

9.7951


In [20]:
log_model_to_mlflow(model, X_test, "BaselineModelPreviousWeek", "mean_absolute_error", score=test_mae)

2025/05/10 21:49:30 INFO mlflow.tracking.fluent: Experiment with name 'BaselineModelPreviousWeek' does not exist. Creating a new experiment.
INFO:src.experiment_utils:Experiment set to: BaselineModelPreviousWeek
INFO:src.experiment_utils:Logged mean_absolute_error: 9.795081967213115
c:\Users\VAISHAK\OneDrive\Desktop\citibikenyc\venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-in

Successfully registered model 'BaselineModelPreviousWeek'.
2025/05/10 21:49:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: BaselineModelPreviousWeek, version 1
Created version '1' of model 'BaselineModelPreviousWeek'.
INFO:src.experiment_utils:Model logged with name: BaselineModelPreviousWeek


🏃 View run stately-dove-344 at: https://dagshub.com/vaishak7026/citibikenyc.mlflow/#/experiments/1/runs/bd459b76c9b74983ade61b155b2a5667
🧪 View experiment at: https://dagshub.com/vaishak7026/citibikenyc.mlflow/#/experiments/1


In [21]:
from sklearn.metrics import mean_absolute_error

test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")


9.7951


In [22]:
import numpy as np
import pandas as pd

class BaselineModelLast4Weeks:
    """
    A baseline model that predicts the average of the last 4 weeks (28 days)
    for each test instance.
    """

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass  # No training needed for baseline

    def predict(self, X_test: pd.DataFrame) -> np.array:
        last_4_weeks_columns = [
            f"rides_t-{7*24}",   # 1 week ago
            f"rides_t-{14*24}",  # 2 weeks ago
            f"rides_t-{21*24}",  # 3 weeks ago
            f"rides_t-{28*24}"   # 4 weeks ago
        ]
        for col in last_4_weeks_columns:
            if col not in X_test.columns:
                raise ValueError(f"Missing required column: {col}")
        return X_test[last_4_weeks_columns].mean(axis=1).to_numpy()


In [23]:
model = BaselineModelLast4Weeks()
predictions = model.predict(X_test)


In [24]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")


8.2787


In [25]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

8.2787


In [26]:
log_model_to_mlflow(model, X_test, "BaselineModelLast4Weeks", "mean_absolute_error", score=test_mae)


2025/05/10 21:49:49 INFO mlflow.tracking.fluent: Experiment with name 'BaselineModelLast4Weeks' does not exist. Creating a new experiment.
INFO:src.experiment_utils:Experiment set to: BaselineModelLast4Weeks
INFO:src.experiment_utils:Logged mean_absolute_error: 8.278688524590164
c:\Users\VAISHAK\OneDrive\Desktop\citibikenyc\venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-intege

Successfully registered model 'BaselineModelLast4Weeks'.
2025/05/10 21:50:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: BaselineModelLast4Weeks, version 1
Created version '1' of model 'BaselineModelLast4Weeks'.
INFO:src.experiment_utils:Model logged with name: BaselineModelLast4Weeks


🏃 View run bedecked-hound-96 at: https://dagshub.com/vaishak7026/citibikenyc.mlflow/#/experiments/2/runs/75b47bbee8b0474c9e28ec361e64113c
🧪 View experiment at: https://dagshub.com/vaishak7026/citibikenyc.mlflow/#/experiments/2


In [27]:
X_test.head()  # Get the total number of rows


,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
219,14,16,19,19,25,46,56,48,29,23,...,0,0,0,4,15,14,13,16,2023-09-01 11:00:00,5329.03
220,45,43,34,49,51,63,63,49,52,29,...,0,0,0,2,7,4,12,32,2023-09-02 11:00:00,5329.03
221,26,32,36,46,62,50,58,48,47,25,...,1,2,0,1,1,12,14,12,2023-09-03 11:00:00,5329.03
222,9,8,19,14,16,24,49,76,49,35,...,0,1,0,1,5,5,17,21,2023-09-04 11:00:00,5329.03
223,8,13,23,15,15,27,66,89,80,50,...,0,0,0,0,0,20,17,12,2023-09-05 11:00:00,5329.03


In [28]:
from src.plot_utils import plot_aggregated_time_series, plot_prediction
plot_aggregated_time_series, plot_prediction

(<function src.plot_utils.plot_aggregated_time_series(features: pandas.core.frame.DataFrame, targets: pandas.core.series.Series, row_id: int, predictions: Optional[pandas.core.series.Series] = None)>,
 <function src.plot_utils.plot_prediction(features: pandas.core.frame.DataFrame, prediction: int)>)